# HMM Assignment
1. Download the dataset hmm_pb1.csv from Canvas. It represents a sequence of
dice rolls $x$ from the Dishonest casino model discussed in class. The model parameters
are exactly those presented in class. The states of $Y$ are 1=’Fair’ and 2=’Loaded’.


#### Import dependencies

In [216]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from os.path import join
from scipy.stats import multivariate_normal
from itertools import repeat
from random import randint

#### Data loading functions

In [217]:
def get_pb1():
    return load_data("hmm_pb1.csv")

def get_pb2():
    return load_data("hmm_pb2.csv")

def load_data(filename):
    path = "data/HMM/"
    data = np.loadtxt(join(path,filename), delimiter=',')
    return data


#### Emission

In [218]:
def fair_die_emission():
    return 1/6

def loaded_die_emission(value):
    if value == 6:
        return .5
    else:
        return .1

def emission(value):
    return np.asarray((fair_die_emission(), loaded_die_emission(value)))

a) Implement the Viterbi algorithm and find the most likely sequence $y$ that generated the observed $x$.
 Use the log probabilities, as shown in the HMM slides from
Canvas. Report the obtained sequence $y$ of 1’s and 2’s for verification. (2 points)

In [219]:
def viterbi(sequence):
    a = np.asarray([[.95,.05],[.05,.95]]) # transition probability
    b = emission(sequence[0]) # Emission probability
    p = np.asarray((.5,.5)) # Start probability
    C = np.ndarray((sequence.size, 2))
    ptr = np.ndarray((sequence.size-1, 2))
    C[0] = np.log(b*p)
    for i in range(1,sequence.size):
        for k in range(2):
            temp = np.log(a[k])+C[i-1]
            C[i,k] = np.log(emission(sequence[i])[k]) + np.max(temp)
            ptr[i-1, k] = np.argmax(temp)
    predicted = np.empty_like(sequence)
    predicted[-1] = np.argmax(C[-1])
    for i in range(predicted.size-2, 0, -1):
        predicted[i] = ptr[i,int(predicted[i+1])]
    predicted = np.round(predicted)+1
    print(predicted)


viterbi(get_pb1())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


b) Implement the forward and backward algorithms and run them on the observed
x. You should memorize a common factor $u_t$ for the $\alpha_t^k$
to avoid floating point underflow, since $\alpha_t^k$ quickly become very small. The same holds for
$\beta_t^k$. Report $\alpha_{125}^1 / \alpha^2_{125}$ and $\beta_{125}^1 / \beta^2_{125}$,
where the counting starts from $t$ = 1. (3 points)

In [220]:
def forward(sequence):
    tran = np.asarray([[.95,.05],[.05,.95]]) # transition probability
    b = emission(sequence[0]) # Emission probability
    p = np.asarray((.5,.5)) # Start probability
    a = np.ndarray((sequence.size, 2))
    a[0] = b*p
    for i in range(1,sequence.size):
        a[i] = emission(sequence[i]) * np.sum(a[i-1]*tran, axis=1)
        a[i] *=6 # multiply by constant to avoid overflow
    return a

a = forward(get_pb1())
print(a[124,0]/a[124,1])

0.3989127511917305


In [221]:
def backward(sequence):
    tran = np.asarray([[.95,.05],[.05,.95]]) # transition probability
    B = np.ndarray((sequence.size, 2))
    B[-1] = np.ones(2)
    for i in range(sequence.size-2, -1, -1):
        B[i] = np.sum(tran*B[i+1]*emission(sequence[i+1]), axis=1)
        B[i] *=6 # multiply by constant to avoid overflow
    return B

B = backward(get_pb1())
print(B[124,0]/B[124,1])

3.856448201261194


2. Download the dataset hmm_pb2.csv from Canvas. It represents a sequence of
10000 dice rolls x from the Dishonest casino model but with other values for the a and
b parameters than those from class. Having so many observations, you are going to
learn the model parameters.

Implement and run the Baum-Welch algorithm using the forward and backward
algorithms that you already implemented for Pb 1. You can initialize the $\pi,a,b$ with
your guess, or with some random probabilities (make sure that $\pi$ sums to 1 and that
$a_{ij}, b^i_k$
sum to 1 for each $i$). The algorithm converges quite slowly, so you might need
to run it for up 1000 iterations or more for the parameters to converge.
Report the values of $\pi,a,b$ that you have obtained. (4 points)


In [222]:
def baum_welch(sequence):
    for _ in range(1000):
        # Expectation step
        a = np.asarray([[.95,.05],[.05,.95]]) # transition probability
        A = forward(sequence)
        B = backward(sequence)
        emissions = np.asarray([emission(i) for i in range(1,7)])
        gamma = A*B / np.sum(A*B, axis=1)[np.newaxis].T+ .0001
        eta = np.ndarray((sequence.size,2,2))
        for t in range(sequence.size-1):
            for i in range(2):
                for j in range(2):
                    temp = A[t,i]*a[i,j]*B[t+1,j]*(emissions[int(sequence[t+1]-1)][j]) /np.sum(A[t,:]*a[i,:]*B[t+1,:]*(emissions[int(sequence[t+1]-1)][:]))
                    eta[t,i,j] = temp

        # maximization step
        pi = gamma[0]
        for i in range(2):
            for j in range(2):
                a[i,j] = np.sum(eta[:,i,j])/np.sum(gamma[:,i])
        for i in range(2):
            for k in range(1,7):
                emissions[k-1,i] = np.sum(gamma[sequence==k, i])/np.sum(gamma[:,i] )
    print("pi", pi)
    print("a", a)
    print("b",emissions)



baum_welch(get_pb2())


pi [0.71487891 0.28532109]
a [[0.99065118, 0.00934882], [0.16993597, 0.83006403]]
b [[0.19826419 0.16067226]
 [0.20423712 0.15047279]
 [0.19011016 0.15381673]
 [0.19726504 0.15244279]
 [0.12517145 0.09942144]
 [0.08495204 0.283174  ]]
